In [1]:
import pandas as pd
from ranking.rank_tfidf import rank_passages_tfidf
from evaluation import evaluate_metrics

df = pd.read_parquet("data/validation_data.parquet")
df["passage_text"] = df.apply(lambda x: x["passages"]["passage_text"], axis=1)
df["is_selected"] = df.apply(lambda x: x["passages"]["is_selected"], axis=1)
df.drop("passages", axis=1, inplace=True)
df.set_index("query_id", inplace=True)
df.head()

,answers,query,query_type,wellFormedAnswers,passage_text,is_selected
query_id,,,,,,
9652,"[Approximately $15,000 per year.]",walgreens store sales average,numeric,[],[The average Walgreens salary ranges from appr...,"[1, 0, 0, 0, 0, 0]"
9653,"[$21,550 per year, The average hourly wage for...",how much do bartenders make,numeric,[],[A bartender’s income is comprised mostly of t...,"[0, 1, 0, 0, 0, 0, 0, 0]"
9654,"[A boil, also called a furuncle, is a deep fol...",what is a furuncle boil,description,[],"[Knowledge center. A boil, also known as a fur...","[0, 0, 0, 0, 0, 0, 1, 0]"
9655,"[Detect and assess a wide range of disorders, ...",what can urinalysis detect,description,[],[Urinalysis: One way to test for bladder cance...,"[0, 0, 0, 0, 1, 0, 0, 0, 0]"
9656,"[Shigellosis, diseases of the nervous system, ...",what is vitamin a used for,description,[],[Since vitamin A is fat-soluble it is not need...,"[0, 0, 0, 0, 1, 0, 0, 0, 0]"


In [2]:
rank_passages_tfidf(df.iloc[0]["query"], df.iloc[0]["passage_text"])

[5, 3, 1, 2, 4, 0]

In [3]:
df["ranked_passages"] = df.apply(lambda x: rank_passages_tfidf(x["query"], x["passage_text"]), axis=1)
df.head()

,answers,query,query_type,wellFormedAnswers,passage_text,is_selected,ranked_passages
query_id,,,,,,,
9652,"[Approximately $15,000 per year.]",walgreens store sales average,numeric,[],[The average Walgreens salary ranges from appr...,"[1, 0, 0, 0, 0, 0]","[5, 3, 1, 2, 4, 0]"
9653,"[$21,550 per year, The average hourly wage for...",how much do bartenders make,numeric,[],[A bartender’s income is comprised mostly of t...,"[0, 1, 0, 0, 0, 0, 0, 0]","[3, 5, 4, 6, 7, 2, 0, 1]"
9654,"[A boil, also called a furuncle, is a deep fol...",what is a furuncle boil,description,[],"[Knowledge center. A boil, also known as a fur...","[0, 0, 0, 0, 0, 0, 1, 0]","[1, 3, 0, 4, 6, 7, 2, 5]"
9655,"[Detect and assess a wide range of disorders, ...",what can urinalysis detect,description,[],[Urinalysis: One way to test for bladder cance...,"[0, 0, 0, 0, 1, 0, 0, 0, 0]","[3, 7, 6, 4, 0, 1, 5, 2, 8]"
9656,"[Shigellosis, diseases of the nervous system, ...",what is vitamin a used for,description,[],[Since vitamin A is fat-soluble it is not need...,"[0, 0, 0, 0, 1, 0, 0, 0, 0]","[4, 0, 7, 8, 5, 6, 3, 1, 2]"


In [4]:
result = evaluate_metrics(df, relevance_col='is_selected', rank_col='ranked_passages')
result

{'MRR': 0.44631403040629675,
 'nDCG': 0.5686959761958269,
 'Precision at 3': 0.19030556384990543,
 'Recall at 3': 0.5204306426462294,
 'MAP': 0.43991272764040756}